### Mobile Robot

[Differential Drive Robots - Columbia University](https://www.cs.columbia.edu/~allen/F17/NOTES/icckinematics.pdf)

In [ ]:
%matplotlib widget
# Dependencies
import matplotlib.animation
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import patches

# Settings
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150

In [ ]:
class MobileRobot(object):
    def __init__(self, ax, pos, w_angle):
        # Parameters and Variables
        self.radius = 0.25
        self.w_radius = 0.5 * self.radius
        self.w_off = np.array([self.radius, 0])
        self.angle = w_angle
        self.pos = pos
        self.w_vels = np.zeros(2)
        self.max_w_vel = 4 * np.pi

        # Plot components
        heading, w_angle, off = self.wheels_offset()
        self.body = patches.Circle(self.pos, self.radius, color=(1,0,0), zorder=10)
        self.wheel_l = patches.Ellipse(self.pos - off, 0.4*self.radius, 2*self.w_radius, angle=w_angle, color=(0,0,1), zorder=10)
        self.wheel_r = patches.Ellipse(self.pos + off, 0.4*self.radius, 2*self.w_radius, angle=w_angle, color=(0,0,1), zorder=10)
        ax.add_patch(self.body)
        ax.add_patch(self.wheel_l)
        ax.add_patch(self.wheel_r)
        self.dir = ax.quiver(self.pos[0], self.pos[1], heading[0], heading[1], scale=40, angles='xy', color=(1,1,1), pivot='middle', zorder=10)
        self.trail = ax.plot(self.pos[0], self.pos[1], linewidth = 1, color='gray', zorder=0)[0]

    def set_w_vels(self, w_vels):
        self.w_vels = np.clip(w_vels, -self.max_w_vel, self.max_w_vel)

    def wheels_offset(self):
        heading = np.array([np.cos(self.angle), np.sin(self.angle)])
        sth = -heading[0]
        cth = heading[1]
        w_angle = np.rad2deg(np.atan2(sth, cth))
        Rot = np.array([[cth, -sth], [sth, cth]])
        offset = Rot @ self.w_off
        return heading, w_angle, offset

    def update(self, dti):
        # Compute kinematics
        vel_l = self.w_vels[0] * self.w_radius
        vel_r = self.w_vels[1] * self.w_radius
        omega = (vel_r - vel_l) / (2*self.radius)
        omega_dti = omega * dti
        if vel_l == vel_r:
            heading_mat = np.array([[np.cos(self.angle), -np.sin(self.angle)], [np.sin(self.angle), np.cos(self.angle)]])
            self.pos += heading_mat @ np.array([vel_l * dti, 0])
        else:
            omega_mat = np.array([[np.cos(omega_dti), -np.sin(omega_dti)], [np.sin(omega_dti), np.cos(omega_dti)]])
            r_rad = self.radius * (vel_r + vel_l) / (vel_r - vel_l)
            icc = np.array([self.pos[0] - r_rad*np.sin(self.angle), self.pos[1] + r_rad*np.cos(self.angle)])
            self.pos = (omega_mat @ (self.pos - icc)) + icc
        self.angle += omega_dti
        
        # Update components
        self.body.set_center(self.pos)
        heading, w_angle, off = self.wheels_offset()
        self.wheel_l.set_center(self.pos-off)
        self.wheel_l.set_angle(w_angle)
        self.wheel_r.set_center(self.pos+off)
        self.wheel_r.set_angle(w_angle)
        self.dir.set_offsets(self.pos)
        self.dir.set_UVC(heading[0], heading[1])
        pt_arr = np.c_[np.array(self.trail.get_data()), self.pos]
        self.trail.set_data(pt_arr[0], pt_arr[1])

In [ ]:
def simple_path_planning(robot, points):
    """ YOUR CODE HERE """
    points_sorted = points[np.argsort(np.linalg.norm(points, axis=1))]
    points_sorted = np.r_[robot.pos.reshape((-1, 2)), points_sorted, robot.pos.reshape((-1, 2))]
    
    mvel = robot.max_w_vel
    vel = mvel * robot.w_radius
    angle = robot.angle
    t_total = 0
    w_vels = []
    stamps = []
    for point_1, point_2 in zip(points_sorted[:-1], points_sorted[1:]):
        pdif = point_2 - point_1
        dist = np.linalg.norm(pdif)
        angle_n = np.atan2(pdif[1], pdif[0]) - angle
        print("Current points:", point_1, point_2)
        print("Distance: %5.3f\nCurrent angle: %5.3f\nOffset angle: %5.3f" % (dist, angle, angle_n))
        w_vel = np.array([-mvel, mvel]) if angle_n > 0 else np.array([mvel, -mvel])
        w_vels.append(w_vel)
        stamps.append(t_total)
        t_total += np.abs(angle_n) * robot.radius / vel
        w_vel = np.array([mvel, mvel])
        w_vels.append(w_vel)
        stamps.append(t_total)
        t_total += dist / vel
        angle += angle_n

    w_vels = np.r_[w_vels,[[0,0]]] 
    stamps = np.r_[stamps,t_total]
    print("Resuls:\n", np.c_[w_vels, stamps])
    return w_vels, stamps

In [ ]:
try:
    ani.event_source.stop()
except (NameError, AttributeError) as e:
    pass

# Plot setup
fig = plt.figure(figsize=(6,3))
fig.set_label("Mobile Robot")
ax = fig.subplots()
ax.set_xlim(-6, 6)
ax.set_ylim(-3, 3)
ax.set_xticks(np.arange(-6, 7, 1))
ax.set_yticks(np.arange(-3, 4, 1))
ax.grid(linewidth=.2)
ax.set_aspect('equal')

# Mobile robot
pos = np.zeros(2).astype(np.float64)
angle = np.random.uniform(-np.pi, np.pi)
robot = MobileRobot(ax, pos, angle)

# Points in space
points = np.random.uniform([-5, -2], [5, 2], (3, 2))
w_vels, stamps = simple_path_planning(robot, points)
ax.plot(points[:,0], points[:,1], '*m', markersize=3)

# Time period
t_f = 20
t = np.linspace(0, t_f, num=t_f*500)
dt = np.diff(t)
dt = np.append(dt, [dt[-1]])
for dti in dt:
    robot.update(dti)

def update(tdt):
    t_i, dt_i = tdt
    for stamp, w_vel in zip(stamps, w_vels):
        if t_i >= stamp:
            robot.set_w_vels(w_vel)
    robot.update(dt_i)

ani = matplotlib.animation.FuncAnimation(fig, update, frames=np.c_[t, dt], interval=1000*np.mean(dt), repeat=False)
fig.tight_layout()
plt.show()